# [Day 18: Snailfish](https://adventofcode.com/2021/day/18)

In [1]:
import typing as tp

## Part 1

In [2]:
def parse(number: str) -> list[tp.Union[str, int]]:
    num_lst = []
    read_int = False
    an_int = 0
    for c in "".join(number.split()):
        if '0' <= c <= '9':
            read_int = True
            an_int = 10 * an_int + int(c)
            continue
        elif read_int:
            num_lst.append(an_int)
            read_int = False
            an_int = 0
        num_lst.append(c)
    return num_lst


def explode(num_lst: list[tp.Union[str, int]], explode_level: int = 5) -> list[tp.Union[str, int]]:
    result = []
    i, level, last_int = 0, 0, -1
    while i < len(num_lst):
        n = num_lst[i]
        # keep track of level
        if n == "[":
            level += 1
        elif n == "]":
            level -= 1

        # explode when explode level reached
        if level >= explode_level:
            # a pair of two num_lsts (integers) will be next: (left, right)
            left, right = num_lst[i+1], num_lst[i+3]
            # left may (or may not) go to the left
            if last_int > 0:
                result[last_int] = result[last_int] + left
            # replace (left, right) the exploded pair by a zero (0)
            result.append(0)
            # right may (or may not) go to the right
            for j in range(i+5, len(num_lst)):
                n = num_lst[j]
                if isinstance(n, int):
                    # integer: it goes to the right
                    n += right
                    result.append(n)
                    # copy the rest and then we're done
                    result.extend(num_lst[j+1:])
                    break
                else:
                    # not integer: just copy
                    result.append(n)
            # done with the while loop
            break
        else:
            # no explode action so far: just copy
            result.append(n)
            # and keep track of last integer
            if isinstance(n, int):
                last_int = i
        i += 1

    return result


def split(num_lst: list[tp.Union[str, int]], split_limit: int=10) -> list[tp.Union[str, int]]:
    result = []
    i = 0
    while i < len(num_lst):
        n = num_lst[i]
        if isinstance(n, int) and (n >= split_limit):
            result.extend(["[", n // 2, ",", (n + 1) // 2, "]"])
            result.extend(num_lst[i+1:])
            break
        else:
            result.append(n)
        i += 1
    return result


def reduce(num_lst: list[tp.Union[str, int]]) -> list[tp.Union[str, int]]:
    before = num_lst
    while True:
        after = explode(before)
        if before == after:
            after = split(before)
            if before == after:
                break
        before = after
    return after


def add(*numbers: str) -> str:
    sub_result = parse(numbers[0])
    for number in numbers[1:]:
        sub_result = reduce(["[", *sub_result, ",", *parse(number), "]"])
    return "".join(map(str, sub_result))


def magnitude(number: str) -> int:
    num_lst = parse(number)
    changes = True
    while changes:
        changes = False
        for i, c in enumerate(num_lst):
            if (c == ",") and isinstance(num_lst[i-1], int) and isinstance(num_lst[i+1], int):
                magnitude = num_lst[i-1] * 3 + num_lst[i+1] * 2
                num_lst = num_lst[:i-2] + [magnitude] + num_lst[i+3:]
                changes = True
                break
    return num_lst[0]

In [3]:
single_explode_examples = (
    ("[[[[[9,8],1],2],3],4]", "[[[[0,9],2],3],4]"),
    ("[7,[6,[5,[4,[3,2]]]]]", "[7,[6,[5,[7,0]]]]"),
    ("[[6,[5,[4,[3,2]]]],1]", "[[6,[5,[7,0]]],3]"),
    ("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]", "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"),
    ("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]", "[[3,[2,[8,0]]],[9,[5,[7,0]]]]"),
)

In [4]:
for number, expected in single_explode_examples:
    print(f"Check Check part 1 (explode ({number!r})): {expected == ''.join(map(str, explode(parse(number))))}")

Check Check part 1 (explode ('[[[[[9,8],1],2],3],4]')): True
Check Check part 1 (explode ('[7,[6,[5,[4,[3,2]]]]]')): True
Check Check part 1 (explode ('[[6,[5,[4,[3,2]]]],1]')): True
Check Check part 1 (explode ('[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]')): True
Check Check part 1 (explode ('[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]')): True


In [5]:
add_examples = {
    1: (
        ("[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]"),
        "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]",
    ),
    2: (
        ("[1,1]", "[2,2]", "[3,3]", "[4,4]"),
        "[[[[1,1],[2,2]],[3,3]],[4,4]]",
    ),
    3: (
        ("[1,1]", "[2,2]", "[3,3]", "[4,4]", "[5,5]"),
        "[[[[3,0],[5,3]],[4,4]],[5,5]]",
    ),
    4: (
        ("[1,1]", "[2,2]", "[3,3]", "[4,4]", "[5,5]", "[6,6]"),
        "[[[[5,0],[7,4]],[5,5]],[6,6]]",
    ),
}

for i, testcase in add_examples.items():
    terms, expected = testcase
    print(f"Check part 1 (add with reduce ({i})): {expected == add(*terms)}")

Check part 1 (add with reduce (1)): True
Check part 1 (add with reduce (2)): True
Check part 1 (add with reduce (3)): True
Check part 1 (add with reduce (4)): True


In [6]:
large_example = {
    "numbers": (
        "[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]",
        "[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]",
        "[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]",
        "[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]",
        "[7,[5,[[3,8],[1,4]]]]",
        "[[2,[2,2]],[8,[8,1]]]",
        "[2,9]",
        "[1,[[[9,3],9],[[9,0],[0,7]]]]",
        "[[[5,[7,4]],7],1]",
        "[[[[4,2],2],6],[8,7]]",
    ),
    "expected": "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]",
}

print(f"Check part 1 (large example): {large_example['expected'] == add(*large_example['numbers'])}")

Check part 1 (large example): True


In [7]:
print(f"Check part 1 (magnitude): {magnitude('[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]') == 4140}")

Check part 1 (magnitude): True


In [8]:
with open(r"..\data\Day 18 input.txt", "r") as fh_in:
    input_data = [row.strip() for row in fh_in.readlines()]
print(f"Input check: {len(input_data) == 100}")

Input check: True


In [9]:
print(f"Answer part 1: {magnitude(add(*input_data))}")

Answer part 1: 3691


## Part 2

In [10]:
def part2(input_data):
    max_magnitude = 0
    for i, left_number in enumerate(input_data):
        for right_number in input_data[i+1:]:
            this_magnitude = magnitude(add(left_number, right_number))
            if this_magnitude > max_magnitude:
                max_magnitude = this_magnitude
            this_magnitude = magnitude(add(right_number, left_number))
            if this_magnitude > max_magnitude:
                max_magnitude = this_magnitude
    return max_magnitude

In [11]:
print(f"Answer part 2: {part2(input_data)}")

Answer part 2: 4756
